In [2]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
#from cartopy import crs
import collections
import cv2
import math

In [7]:
class annotator:
    def __init__(self, sub_directory):
        self.sub_directory = sub_directory
        
    def state_dcc_directory(self, dcc_directory):
        self.dcc_directory = dcc_directory    
        
    def number_of_tiles(self, num_tiles):
        self.num_tiles = num_tiles
        
    def get_tile_urls(self, tile_name_tile_url_unlabeled):
        """
        self.tile_name_tile_url_unlabeled: npy array of the initial tiles that have not been labeled
        self.tile_name_tile_url_tiles_for_annotators: npy array of the tiles to be allocated to the annotator 
        """
        
        self.tile_name_tile_url_unlabeled = np.load(tile_name_tile_url_unlabeled) #the tiles that have not yet been labeled to date
        print("Unlabeled Tiles",self.tile_name_tile_url_unlabeled.shape)
        self.tile_name_tile_url_tiles_for_annotators = self.tile_name_tile_url_unlabeled[range(self.num_tiles),:] #create an array of the tiles that will be allocated to this annotator
        
        self.tile_url = self.tile_name_tile_url_tiles_for_annotators[:,1] #get the urls of the tiles that will allocated to the annotator
    
    def track_tile_annotations(self, tile_name_tile_url_labeled):
        """
        self.tile_name_tile_url_remaining: npy array of the remaining tiles to be annotated; this will then be passed in the next iteration
        self.tile_name_tile_url_labeled: npy array of the tiles labeled
        """
        
        self.tile_name_tile_url_labeled = np.load(tile_name_tile_url_labeled) #the tiles that have not yet been labeled to date
        self.tile_name_tile_url_labeled = np.concatenate((self.tile_name_tile_url_labeled, self.tile_name_tile_url_tiles_for_annotators), axis=0)
        print("Labeled Tiles", self.tile_name_tile_url_labeled.shape)

        self.tile_name_tile_url_remaining = np.delete(self.tile_name_tile_url_unlabeled, range(self.num_tiles), 0) #the numpy array of the remaining tiles 
                                                                                  #(remove the tiles that the annotator is labeling)
        print(self.tile_name_tile_url_remaining.shape)
   
        if len(self.tile_name_tile_url_tiles_for_annotators) + len(self.tile_name_tile_url_remaining) != len(self.tile_name_tile_url_unlabeled):
            raise Exception("The number of remaining tiles and the tiles allocated to annotaters is less \
                             than the number of tiles passed through this function")
    
    def make_subdirectories(self):
        self.new_dir = self.dcc_directory + "/" + self.sub_directory
        
        os.makedirs(self.new_dir, exist_ok = True)

        self.tiles_dir = os.path.join(self.new_dir,'tiles') #directory for the naip data 
        os.makedirs(self.tiles_dir,exist_ok=True)

        self.chips_dir = os.path.join(self.new_dir,'chips') #directory to hold chips that are clipped from naip tiles
        os.makedirs(self.chips_dir,exist_ok=True)

        self.chips_positive_dir = os.path.join(self.new_dir,'chips_positive') #directory to hold chips with tanks
        os.makedirs(self.chips_positive_dir,exist_ok=True)

        self.chips_negative_dir = os.path.join(self.new_dir,'chips_negative') #directory to hold chips with tanks
        os.makedirs(self.chips_negative_dir,exist_ok=True)

        self.chips_xml_dir = os.path.join(self.new_dir,'chips_positive_xml') #directory to hold xml files
        os.makedirs(self.chips_xml_dir,exist_ok=True)
        
        #Make directory to store all xml after correction
        self.chips_positive_corrected_xml_dir = os.path.join(self.new_dir,"chips_positive_corrected_xml")
        os.makedirs(self.chips_positive_corrected_xml_dir, exist_ok = True)
    
    def download_images(self):
        destination_of_filenames = [] #use so that we can index over the file names for processing later
        for i in range(self.num_tiles):
            print(i)
            destination_of_filenames.append(download_url(self.tile_url[i], self.tiles_dir,
                                                         progress_updater=DownloadProgressBar()))
        return destination_of_filenames
    
    def tile_rename(self):
        """Rename all the tiles into the standard format outlined in repo readme 
        """

        self.tile_names = os.listdir(self.tiles_dir) #get a list of all of the tiles in tiles directory
        print(self.tile_names)
        
        for tile_name in self.tile_names: 
            tile_name_split = tile_name.split('_')
            old_tile_path = os.path.join(self.tiles_dir, tile_name)
            new_tile_path = os.path.join(self.tiles_dir, tile_name_split[6]+'_'+tile_name_split[7]+'_'+tile_name_split[8]+'_'+tile_name_split[9]+'_'+  \
                                                    tile_name_split[10]+'_'+tile_name_split[11]+'_'+tile_name_split[12]+'_'+tile_name_split[13]+'_'+  \
                                                    tile_name_split[14]+'_'+tile_name_split[15].split(".")[0]+".tif")

            if os.path.isfile(new_tile_path):
                print('Bypassing download of already-downloaded file {}'.format(os.path.basename(new_tile_path)))
            
            else:
                os.rename(old_tile_path, new_tile_path)

    def chip_tiles(self):
        """Segment tiles into 512 x 512 pixel chips, preserving resolution
        """
        print("function_opened")
        self.tile_names = os.listdir(self.tiles_dir) #get a list of all of the tiles in tiles directory
        for tile_name in self.tile_names: #index over the tiles in the tiles_dir 
            file_name, ext = os.path.splitext(tile_name) # File name
            print(tile_name)
            item_dim = int(512)
            box_list = []
            count = 1            
            tile = cv2.imread(os.path.join(self.tiles_dir, tile_name)) 
            tile_height,  tile_width,  tile_channels = tile.shape
            tile_height,  tile_width,  tile_channels = math.ceil(tile_height), math.ceil(tile_width), math.ceil(tile_channels)
                        
            col_index = int(tile_width/512)
            row_index = int(tile_width/512)
            
            for x in range(0,row_index):
                for y in range(0, col_index):
                
                    #box = 
                    chip_img = tile[y*item_dim:y*item_dim+item_dim, x*(item_dim):x*(item_dim)+item_dim]

                    #At the end of the image
                    if chip_img.width != 512:
                        black_width = 512 - chip_img.width
                        black_height = 512 - chip_img.height
                        black_image = np.zeros((black_height,black_width,3), np.uint8)
                        print("Incorrect Width")
                        print(black_img.width)
                        print(black_img.height)
                        chip_img = np.concatenate([chip_img, blank_image])

                    if chip_img.height != 512:
                        black_width = 512 - chip_img.width
                        black_height = 512 - chip_img.height
                        black_image = np.zeros((black_height,black_width,3), np.uint8)
                        print("Incorrect Height")
                        print(black_img.width)
                        print(black_img.height)
                        chip_img = np.concatenate([chip_img, blank_image])
                        
                    chips_save_path = os.path.join(self.chips_dir, file_name+ '_'+ \
                               str(count).zfill(6) + '.jpg') 
                    # The index is a six-digit number like '000023'. 
                    cv2.imwrite(os.path.join(chips_save_path), cropped)
                    count += 1
            print(count)


In [ ]:
image = cv2.imread(r"C:/chips_allocation/Check/tiles/ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif") 
y=0
x=512*19
h=512
w=512
#cropped = image[x:w, y:h]
cropped = image[y:y+h, x:x+w]
print(image.shape)

# Filename
filename = 'test2.jpg'
cv2.imshow("Cropped", cropped)
cv2.waitKey(0)

In [20]:
image = cv2.imread(r"C:/chips_allocation/Check/tiles/ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif") 
print(image.shape[0])
h,w,d=image.shape
print(h/512)


12220
23.8671875


In [4]:
parent_directory = "C:\chips_allocation"
annotation_directory = "Check"
number_of_tiles =10
tiles_remaining = "tile_name_tile_url_remaining_expanded.npy"
tiles_labeled = "tile_name_tile_url_labeled.npy"

In [5]:
dist = ap.annotator(annotation_directory) #create the processing class
dist.state_dcc_directory(parent_directory)
dist.number_of_tiles(number_of_tiles)
dist.get_tile_urls(tiles_remaining)
dist.make_subdirectories()
#dist.tile_names
#dist.download_images()
#dist.tile_rename()

Unlabeled Tiles (801, 2)


In [ ]:
dist.chip_tiles()

function_opened
ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif
12220 9770
23 19
24 20
481
ca_060cm_2018_38122_m_3812263_ne_10_060_20180723.tif
12150 9750
23 19
24 20
481
ct_060cm_2018_42072_m_4207260_se_18_060_20180823.tif
12817 9958
25 19
26 20
521
ky_060cm_2018_37087_m_3708712_nw_16_060_20180725.tif
12523 10311
24 20
25 21
526
mi_060cm_2018_43083_m_4308317_sw_17_060_20180703.tif
12965 9885
25 19
26 20
521
nm_060cm_2018_32103_m_3210330_nw_13_060_20180618.tif
12210 10470
23 20
24 21
505
tx_060cm_2018_30101_m_3010109_ne_14_060_20190105.tif
12320 10780
24 21
25 22
551
tx_060cm_2018_32095_m_3209516_se_15_060_20181120.tif
12250 10490
23 20
24 21
505
ut_060cm_2018_40111_m_4011109_se_12_060_20180828.tif
12160 9420
23 18
24 19
457
wi_060cm_2018_46092_m_4609224_ne_15_060_20181022.tif
12674 9105
24 17
25 18
451


In [19]:
tile.shape[0] #Height


12220